In [39]:
import pandas as pd
import numpy as np
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
def lag_data(data, lag=1):
    data = list(data)
    X, y = [], []
    for i in range(len(data) - lag - 1):
        a = data[i: i + lag]
        X.append(a)
        y.append(data[i + lag])
    return np.array(X), np.array(y)

In [35]:
def create_simple_network(lags):
    model = Sequential()
    model.add(LSTM(lags, input_dim=lags))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [4]:
data = pd.read_csv("../data/Sample/2_arrivals.csv", header=None)[1]
data.head()

0    12
1    44
2    38
3    21
4    20
Name: 1, dtype: int64

In [70]:
lags = 7
scaler = MinMaxScaler(feature_range=(0, 1))
model = create_simple_network(lags)
data = pd.read_csv("../data/Sample/2_arrivals.csv", header=None)[1]
data = scaler.fit_transform(data)
train, test = train_test_split(data, test_size=0.30)
X_train, y_train = lag_data(train, lag=lags)
X_test, y_test = lag_data(test, lag=lags)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

model.fit(X_train, y_train, nb_epoch=100, batch_size=128, verbose=0)
train_preds = model.predict(X_train)

/Users/computer/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/computer/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/computer/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [71]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

train_preds = scaler.inverse_transform(train_preds)
y_train = scaler.inverse_transform(y_train)
test_preds = scaler.inverse_transform(test_preds)
y_test = scaler.inverse_transform(y_test)

train_score = math.sqrt(mean_squared_error(y_train, train_preds))
print('Train Score: %.2f RMSE' % (train_score))
test_score = math.sqrt(mean_squared_error(y_test, test_preds))
print('Test Score: %.2f RMSE' % (test_score))

Train Score: 57.77 RMSE
Test Score: 54.60 RMSE


/Users/computer/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:377: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/computer/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:377: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
